In [15]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go

# Load the datasets
gold_data = pd.read_excel('gold.xlsx')
btc_data = pd.read_excel('btc.xlsx')

# Convert the Date columns to datetime format for both datasets
gold_data['Date'] = pd.to_datetime(gold_data['Date'])
btc_data['Date'] = pd.to_datetime(btc_data['Date'])

# Calculate daily returns for both gold and bitcoin
gold_data['Gold_Return'] = gold_data['Close'].pct_change()
btc_data['BTC_Return'] = btc_data['btc_rial'].pct_change()

# Merge the data on the Date column
merged_data = pd.merge(gold_data[['Date', 'Gold_Return']], btc_data[['Date', 'BTC_Return']], on='Date', how='inner')

# Drop NaN values that may have resulted from pct_change
merged_data.dropna(inplace=True)

# Calculate mean returns and covariance matrix
mean_returns = merged_data[['Gold_Return', 'BTC_Return']].mean()
cov_matrix = merged_data[['Gold_Return', 'BTC_Return']].cov()

# Define portfolio performance function
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns * weights)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return returns, volatility

# Define the objective function to minimize (negative Sharpe ratio)
def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.0):
    p_returns, p_volatility = portfolio_performance(weights, mean_returns, cov_matrix)
    sharpe_ratio = (p_returns - risk_free_rate) / p_volatility
    return -sharpe_ratio

# Constraints and bounds
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
bounds = tuple((0, 1) for _ in range(len(mean_returns)))

# Initial guess (equal weights)
init_guess = len(mean_returns) * [1. / len(mean_returns)]

# Perform optimization
optimal_result = minimize(negative_sharpe_ratio, init_guess, args=(mean_returns, cov_matrix), method='SLSQP', bounds=bounds, constraints=constraints)

# Get the optimal weights
optimal_weights = optimal_result.x

# Calculate the optimal portfolio performance
optimal_returns, optimal_volatility = portfolio_performance(optimal_weights, mean_returns, cov_matrix)

# Generate portfolios for efficient frontier
num_portfolios = 10000
results = np.zeros((3, num_portfolios))
for i in range(num_portfolios):
    weights = np.random.random(len(mean_returns))
    weights /= np.sum(weights)
    p_returns, p_volatility = portfolio_performance(weights, mean_returns, cov_matrix)
    results[0,i] = p_volatility
    results[1,i] = p_returns
    results[2,i] = (p_returns - 0) / p_volatility  # Sharpe ratio with zero risk-free rate

# Create a DataFrame for visualization
df = pd.DataFrame({
    'Volatility': results[0],
    'Return': results[1],
    'Sharpe Ratio': results[2]
})

# Plot using Plotly
fig = px.scatter(df, x='Volatility', y='Return', color='Sharpe Ratio', 
                 title='Efficient Frontier with Optimal Portfolio', 
                 labels={'Volatility': 'Volatility (Risk)', 'Return': 'Expected Return'},
                 template='plotly_dark', 
                 hover_data={'Volatility': ':.2f', 'Return': ':.2f', 'Sharpe Ratio': ':.2f'})

# Add optimal portfolio point
fig.add_trace(go.Scatter(
    x=[optimal_volatility],
    y=[optimal_returns],
    mode='markers',
    marker=dict(color='red', size=15, symbol='star'),
    name='Optimal Portfolio'
))

# Customize layout
fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    title_x=0.5,
    font=dict(size=14)
)

# Show the plot
fig.show()

# Display the results
print("Optimal Weights for Portfolio:")
print(f"Gold: {optimal_weights[0]:.2%}")
print(f"Bitcoin: {optimal_weights[1]:.2%}")

print("\nOptimal Portfolio Performance:")
print(f"Expected Daily Return: {optimal_returns:.2%}")
print(f"Daily Volatility (Risk): {optimal_volatility:.2%}")

Optimal Weights for Portfolio:
Gold: 77.41%
Bitcoin: 22.59%

Optimal Portfolio Performance:
Expected Daily Return: 0.20%
Daily Volatility (Risk): 1.74%
